In [163]:
from gensim import corpora, models, similarities
documents = ["Human machine interface for lab abc computer applications",
            "A survey of user opinion of computer system response time",
            "The EPS user interface management system",
            "System and human system engineering testing of EPS",
            "Relation of user perceived response time to error measurement",
            "The generation of random binary unordered trees",
            "The intersection graph of paths in trees",
            "Graph minors IV Widths of trees and well quasi ordering",
            "Graph minors A survey"]

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

''' 
remove words that appear only once
1> Calculate the frequency for each word in entire document
2> Remove words with only one frequency
E.g. "abc", "lab" in first sentence will be removed
'''
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
    for text in texts]

from pprint import pprint   # pretty-printer
pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [164]:
dictionary = corpora.Dictionary(texts)
dictionary.save('/tmp/deerwester.dict') # store the dictionary, for future reference
print(dictionary.token2id)

{u'minors': 11, u'graph': 10, u'system': 6, u'trees': 9, u'eps': 8, u'computer': 1, u'survey': 5, u'user': 7, u'human': 2, u'time': 4, u'interface': 0, u'response': 3}


In [165]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('/tmp/deerwester.mm', corpus) # store to disk, for later use
print(corpus)

[[(0, 1), (1, 1), (2, 1)], [(1, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(0, 1), (6, 1), (7, 1), (8, 1)], [(2, 1), (6, 2), (8, 1)], [(3, 1), (4, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(5, 1), (10, 1), (11, 1)]]


In [166]:
# convert tokenized documents to vectors
new_doc = "bunary randam trees unorder"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec) # the word "interaction" does not appear in the dictionary and is ignored

lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)
vec_lsi = lsi[new_vec] # convert the query to LSI space
print(vec_lsi)

[(9, 1)]
[(0, -0.012746183038294849), (1, 0.49016179245310376)]


In [167]:
index = similarities.MatrixSimilarity(lsi[corpus]) # transform corpus to LSI space and index it

In [168]:
sims = index[vec_lsi] # perform a similarity query against the corpus
print(list(enumerate(sims))) # print (document_number, document_similarity) 2-tuples

[(0, -0.18518142), (1, 0.22892301), (2, -0.17928416), (3, -0.28446588), (4, 0.30398417), (5, 0.99999994), (6, 0.99983984), (7, 0.99967402), (8, 0.98480445)]


In [169]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims) # print sorted (document number, similarity score) 2-tuples

[(5, 0.99999994), (6, 0.99983984), (7, 0.99967402), (8, 0.98480445), (4, 0.30398417), (1, 0.22892301), (2, -0.17928416), (0, -0.18518142), (3, -0.28446588)]


In [170]:
sims_list = list(enumerate(sims))
for i in range(0, len(sims_list)):
    print(str(sims_list[i][1][1] * 100) + " : " + documents[sims_list[i][1][0]])

99.9999940395 : The generation of random binary unordered trees
99.9839842319 : The intersection graph of paths in trees
99.9674022198 : Graph minors IV Widths of trees and well quasi ordering
98.4804451466 : Graph minors A survey
30.3984165192 : Relation of user perceived response time to error measurement
22.8923007846 : A survey of user opinion of computer system response time
-17.9284155369 : The EPS user interface management system
-18.5181424022 : Human machine interface for lab abc computer applications
-28.4465879202 : System and human system engineering testing of EPS
